# Simulation

Dans cette partie j'ai un peu reporté les concepts de la partie "essai" car j'étais un peu triste au début de ne pas avoir eu d'assez beaux clusters.

In [ ]:
import pandas as pd
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.decomposition import PCA, KernelPCA
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
import umap
from src.view import view_clusters, view_projection
import numpy as np
import squarify
import math
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import DistanceMetric
from scipy import stats
from src.dataset import get_data_until
from sklearn.metrics import adjusted_rand_score
from tqdm import tqdm

plt.style.use("ggplot")

In [ ]:
processed = Path("data/processed")

customers = pd.read_csv(processed / "unique_customer_orders2.csv")
customers.head()

On va déjà plotter la RFM pour voir s'il y a une différence par rapport à avant

In [ ]:
RFM = ["recency", "amount", "number_of_orders"]

In [ ]:
customers[RFM]["recency"]

In [ ]:
customers_rfm = customers[RFM]


fig = px.scatter_3d(x=customers_rfm[RFM[0]], y=customers_rfm[RFM[1]], z=customers_rfm[RFM[2]], opacity=0.3, color=customers["wealthy"])
fig.update_layout(
    width=1400,
    height=800,
    scene={f"{a}axis": {"title":{"text":f"{t} ({a})"}} for a,t in zip(["x","y","z"], RFM)}
)

On peut donc essayer de rajouter des features au fur et à mesure parmis celles extraites

On va commencer par scaler et encoder nos données si besoin

In [ ]:
encoder = LabelEncoder()

customers["frequent_cat"] = encoder.fit_transform(customers["frequent_cat"])
customers.head()

In [ ]:
customers[["amount"]].describe()

In [ ]:
customers[["recency"]].describe()

In [ ]:
plt.title("Pairplot for some features and RFM")
sns.pairplot(customers[RFM + ["respected_ratio", "estimation_error", "freight_value", "review_score"]])

On peut maintenant refaire notre clusering avec les nouvelles features pour voir si elles aides à identifier des clusters.

Plus tard on s'occupera de regarder les outliers.

In [ ]:
cols = ["delivery_delay", "estimation_error", "number_of_orders", "respected_ratio", "lat", "lng", "freight_value", "price", "review_answer_delay", "review_score", "review_level"]

In [ ]:
X_scaled = StandardScaler().fit_transform(customers[RFM + cols])
X_scaled.shape

In [ ]:
models =[
    PCA(random_state=0),
    KernelPCA(kernel="rbf", random_state=0),
    *[TSNE(perplexity=p, n_jobs=-1, random_state=0) for p in np.logspace(-1, 2, num=4)*3]
]

In [ ]:
view_projection(models, X_scaled, hue=customers["wealthy"], p=0.04)

In [ ]:
view_projection(models, X_scaled, hue=customers["frequent_cat"], p=0.04)

In [ ]:
customers.describe()

In [ ]:
X_scaled = StandardScaler().fit_transform(customers[RFM + ["respected_ratio", "freight_value", "price", "review_score", "review_level", "estimation_error", "delivery_delay", "number_of_orders"]])

Ici j'essaye plein de méthodes de façons de projeter mes données pour voir si c'est concluant d'où l'utilisation de ma fonction view_projection

In [ ]:
view_projection(models, X_scaled, hue=customers["wealthy"], p=0.04)

In [ ]:
view_projection([*[umap.UMAP(n_neighbors=n, random_state=0) for n in range(2,20, 4)]], X_scaled, hue=customers["wealthy"], p=0.05)

In [ ]:
view_projection([TSNE(n_iter=2000, perplexity=30, random_state=0), TSNE(n_iter=2000, perplexity=300, random_state=0)], X_scaled, hue=customers["wealthy"], p=0.05)

Je stock donc mes meilleurs projections

In [ ]:
favorite_projectors = [
    TSNE(n_iter=1200, perplexity=30, random_state=0, n_jobs=-1),
    umap.UMAP(n_neighbors=16, random_state=0, n_jobs=-1)
]

In [ ]:
for projector in favorite_projectors:
    y = projector.fit_transform(X_scaled[:5000])

    visualizer = SilhouetteVisualizer(KMeans(n_init="auto"), k=(4,20), timings=False)
    visualizer.fit(y)
    visualizer.poof()

In [ ]:
for projector in favorite_projectors:
    print(type(projector).__name__)

    y = projector.fit_transform(X_scaled[:1000])

    plt.figure()
    visualizer = KElbowVisualizer(KMeans(n_init="auto", random_state=0), k=(4,20), timings=False)
    visualizer.fit(y)
    visualizer.show()

    k = visualizer.elbow_value_

    model = KMeans(n_clusters=k, random_state=0)
    labels = model.fit_predict(y)
    
    fig, ax = plt.subplots()
    sns.scatterplot(x=y.T[0], y=y.T[1], hue=labels, ax=ax)

In [ ]:
for projector in favorite_projectors:
    print(type(projector).__name__)

    y = projector.fit_transform(X_scaled)

    plt.figure()
    visualizer = KElbowVisualizer(KMeans(n_init="auto", random_state=0), k=(4,20), timings=False, n_jobs=-1)
    visualizer.fit(y)
    visualizer.show()

    k = visualizer.elbow_value_

    model = KMeans(n_clusters=k, random_state=0)
    labels = model.fit_predict(y)
    
    fig, ax = plt.subplots()
    sns.scatterplot(x=y.T[0], y=y.T[1], hue=labels, ax=ax)

Je vais donc revenir aux bases et faire cela avec le moins de features possible pour rester dans un point de vue métier.

Ici on peut très bien critiquer mon travail en disant qu'il y a trop de features qui brouillent les clients intéressant des non intéressants.

In [ ]:
customers.columns

In [ ]:
# Voici les colonnes que j'ai choisis
cols = RFM
X = customers[cols]
X_scaled = StandardScaler().fit_transform(X)

In [ ]:
for projector in favorite_projectors:
    print(type(projector).__name__)

    y = projector.fit_transform(X_scaled)

    plt.figure()
    visualizer = KElbowVisualizer(KMeans(n_init="auto", random_state=0), k=(2,14), timings=False, n_jobs=-1)
    visualizer.fit(y)
    visualizer.show()

    k = visualizer.elbow_value_

    model = KMeans(n_clusters=k, random_state=0)
    labels = model.fit_predict(y)
    
    fig, ax = plt.subplots()
    sns.scatterplot(x=y.T[0], y=y.T[1], hue=labels, ax=ax)

In [ ]:
y = umap.UMAP(n_neighbors=3, n_components=3, random_state=0, n_jobs=-1).fit_transform(X_scaled)

px.scatter_3d(x=y.T[0], y=y.T[1], z=y.T[2], opacity=0.3)

In [ ]:
model = KElbowVisualizer(KMeans(random_state=0))
model.fit(X_scaled)
model.show()

In [ ]:
model = KMeans(n_clusters=4, n_init="auto")
model.fit(X_scaled)

In [ ]:
px.scatter_3d(x=X_scaled.T[0], y=X_scaled.T[1], z=X_scaled.T[2], color=model.labels_, opacity=0.3)

# Itération 2

J'ai l'impression que certaines analyses ne servent à rien. Je vais essayer d'utiliser une méthode RFM que j'ai vu sur Kaggle qui consiste à créer à partir des données une pré-segmentation pour savoir si on doit oui ou non être préocupper par un client ce qui nous permet de connaître et différencier les bons des mauvais clients

In [ ]:
customers = pd.read_csv(processed / "unique_customer_orders3.csv")
customers.head()

In [ ]:
customers["segment"].describe()

On peut donc refaire les clustering intéressants mais en coloriant par la segmentation

In [ ]:
customers["segment"].value_counts()

In [ ]:
count = customers["segment"].value_counts()
norm = count / count.sum()
labels = pd.Series(count.index).str.cat(" ( " + (norm * 100).round(2).astype(str).values + "% )")
squarify.plot(count, label=labels, color = ['gold', 'teal', 'steelblue', 'limegreen', 'darkorange', 'coral'])

In [ ]:
X_scaled = StandardScaler().fit_transform(customers[RFM])
X_scaled

In [ ]:
view_projection(favorite_projectors, X_scaled, hue=customers["segment"])

On remarque que les clients fidèles sont bien éparpillés dans les clusters

C'est déjà pas mal. Voici le résultat d'une étude du meilleur K avec KMeans

In [ ]:
model = KMeans(random_state=0, n_init="auto")

visualizer = KElbowVisualizer(model, k=range(2,14), timings=False)
visualizer.fit(X_scaled)
visualizer.show();

In [ ]:
customers

In [ ]:
for cols in [["frequency"],
             ["frequency", "freight_value"],
             ["frequency", "review_level", "review_score"],
             ["delivery_delay"],
             ["frequency", "review_level", "review_score", "freight_value", "delivery_delay", "respected_ratio"]]:
    X_scaled = StandardScaler().fit_transform(customers[RFM + cols])
    view_projection(favorite_projectors, X_scaled, hue=customers["segment"])

    plt.figure()
    model = KMeans(random_state=0, n_init="auto")
    visualizer = KElbowVisualizer(model, k=range(2,10), timings=False)
    visualizer.fit(X_scaled)
    visualizer.show()

In [ ]:
# colonnes sélectionnées
data = customers

cols = ["recency", "amount"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data[cols])

model = KMeans(n_init="auto", n_clusters=4, random_state=0)
model.fit(X_scaled)

customers_true_labels = model.labels_

In [ ]:
scores = []

for m in tqdm(range(1,25)):
    # On perd des clients qu'on avait auparavant
    data_m = get_data_until(month=m)

    # Les clients correspondant dans 'customers'
    customers_m = customers[customers.customer_unique_id.isin(data_m.customer_unique_id)]
    customers_labels = customers_true_labels[customers_m.index]

    # On transform avec le même scaler
    X_scaled = scaler.transform(data_m[cols])

    model.fit(X_scaled)

    # On récupère le score ARI
    scores.append(adjusted_rand_score(customers_labels, model.labels_))

scores

In [ ]:
plt.title("ARI scores per month difference")
plt.plot(scores)

In [ ]:
scores = []

for d in tqdm(range(30, 90, 7)):
    # On perd des clients qu'on avait auparavant
    data_m = get_data_until(days=d)

    # Les clients correspondant dans 'customers'
    customers_m = customers[customers.customer_unique_id.isin(data_m.customer_unique_id)]
    customers_labels = customers_true_labels[customers_m.index]

    # On transform avec le même scaler
    X_scaled = scaler.transform(data_m[cols])

    model.fit(X_scaled)

    # On récupère le score ARI
    scores.append((d, adjusted_rand_score(customers_labels, model.labels_)))

In [ ]:
for (d, score) in scores:
    print(f"{d=} ; {score=}")

In [ ]:
X = np.array(scores)
days = X[:, 0]
scores_d = X[:, 1]

plt.title("ARI scores per day difference")
plt.plot(days, scores_d)

In [ ]:
scores = []

for d in tqdm(range(50, 65, 1)):
    # On perd des clients qu'on avait auparavant
    data_m = get_data_until(days=d)

    # Les clients correspondant dans 'customers'
    customers_m = customers[customers.customer_unique_id.isin(data_m.customer_unique_id)]
    customers_labels = customers_true_labels[customers_m.index]

    # On transform avec le même scaler
    X_scaled = scaler.transform(data_m[cols])

    model.fit(X_scaled)

    # On récupère le score ARI
    scores.append((d, adjusted_rand_score(customers_labels, model.labels_)))

X = np.array(scores)
days = X[:, 0]
scores_d = X[:, 1]

plt.title("ARI scores per day difference")
plt.plot(days, scores_d)

In [ ]:
RM = ["recency", "amount"]

In [ ]:
X2 = get_data_until(days=56)

In [ ]:
X1 = customers[customers.customer_unique_id.isin(X2.customer_unique_id)].reset_index(drop=True)

In [ ]:
X2

In [ ]:
X1

In [ ]:
(X1.customer_unique_id == X2.customer_unique_id).value_counts()

In [ ]:
X1[RM].describe()

In [ ]:
X2[RM].describe()

In [ ]:
cols = ['recency', 'delivery_delay', 'estimation_error',
       'number_of_orders', 'respected_ratio', 'amount', 'lat', 'lng',
       'frequency', 'freight_value', 'review_answer_delay',
       'review_score', 'review_level']

scaler = StandardScaler()

In [ ]:
corr = customers[cols].corr()

sns.heatmap(corr)

In [ ]:
customers.describe()

In [ ]:
def show_dists(df):
    import math
    l = math.ceil(len(df.columns) / 3)

    fig, axs = plt.subplots(l, 3, figsize=(20,l*5))

    axs = axs.flatten()

    for ax, col in zip(axs, df.columns):
        sns.histplot(df[col], ax=ax)

In [ ]:
customers.describe()

In [ ]:
show_dists(customers[["recency", "delivery_delay", "estimation_error", "respected_ratio", "number_of_orders", "amount", "frequency", "freight_value", "review_answer_delay", "review_score", "review_level"]])

In [ ]:
log_cols = ["delivery_delay", "amount", "freight_value", "review_answer_delay", "number_of_orders"]

customers[[x + "_log" for x in log_cols]] = customers[log_cols].apply(lambda x: x+1e-8).apply(np.log)
customers

In [ ]:
show_dists(customers[np.array(list(zip(log_cols, [x + "_log" for x in log_cols]))).flatten()])

In [ ]:
customers[[x + "_log" for x in log_cols]].describe()

In [ ]:
all_cols = cols + [x + "_log" for x in log_cols]
std_cols = [x + "_std" for x in all_cols]
 
customers[std_cols] = scaler.fit_transform(customers[all_cols])
customers.columns

In [ ]:
from sklearn.preprocessing import MinMaxScaler

all_cols = all_cols + std_cols

mm_cols = [x + "_mm" for x in all_cols]

customers[mm_cols] = MinMaxScaler().fit_transform(customers[all_cols])
customers.columns

In [ ]:
use_cols = ['amount_log_std_mm', 'number_of_orders_log_std_mm', 'recency_std_mm', 'respected_ratio_std_mm', 'freight_value_log_std_mm']

In [ ]:
subset = customers

In [ ]:
viz = KElbowVisualizer(KMeans(random_state=0, n_init="auto"), k=(2,12), timings=False)
viz.fit(subset[use_cols])
viz.show();

In [ ]:
#projector = TSNE(n_components=2, n_jobs=-1, random_state=0, perplexity=120, learning_rate=100)

dists = ["euclidean", "chebyshev", "correlation", "hellinger"]

for neighbor in tqdm([5, 10, 20, 30, 40, 50]):
    fig, axs = plt.subplots(math.ceil(len(dists) / 2), 1 if len(dists) == 1 else 2, figsize=(20,20))
    axs = axs.flatten()
    for ax, dist in tqdm(zip(axs, dists)):
        ax.set_title(dist)
        projector = umap.UMAP(n_neighbors=neighbor, random_state=0, n_jobs=-1, repulsion_strength=1.2, metric=dist)
        projector.fit(subset[use_cols])
        y = projector.embedding_.T
        sns.scatterplot(x=y[0], y=y[1], hue=subset["segment"], ax=ax)
    plt.show(fig)

In [ ]:
selected = ['amount_log_std_mm', 'number_of_orders_log_std_mm', 'recency_std_mm', 'respected_ratio_std_mm', 'freight_value_log_std_mm']

projector = umap.UMAP(n_neighbors=50, random_state=0, n_jobs=-1, repulsion_strength=1.2)

model = KMeans(random_state=0, n_init="auto", n_clusters=3)
model.fit(customers[selected])

projector.fit(customers[selected])

y = projector.embedding_.T

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(20,10))
axs = axs.flatten()

sns.scatterplot(x=y[0], y=y[1], hue=customers["segment"], ax=axs[0])
sns.scatterplot(x=y[0], y=y[1], hue=model.labels_, ax=axs[1])

In [ ]:
viz = KElbowVisualizer(KMeans(random_state=0, n_init="auto"), k=(2,12), timings=False)
viz.fit(y.T)
viz.show();

In [ ]:
model = DBSCAN(n_jobs=-1)
model.fit(y.T)
sns.scatterplot(x=y[0], y=y[1], hue=model.labels_)

# Modèle fixe et maintenance

In [ ]:
model = SpectralClustering(n_clusters=4, n_jobs=-1)
y_pred = y[:,:500]
print(y_pred.shape)
model.fit(y_pred.T)
sns.scatterplot(x=y_pred[0], y=y_pred[1], hue=model.labels_)

In [ ]:
dist = DistanceMetric.get_metric('euclidean')
knn = NearestNeighbors(n_neighbors=model.n_neighbors, metric='precomputed')
knn.fit(model.affinity_matrix_)

distances, indices = knn.kneighbors(dist.pairwise([[1,1]], y_pred.T))

sns.scatterplot(x=y_pred[0], y=y_pred[1], hue=model.labels_)
sns.scatterplot(x=[1], y=[1])

In [ ]:
dist = DistanceMetric.get_metric('euclidean')
knn = NearestNeighbors(n_neighbors=model.n_neighbors, metric='precomputed')
knn.fit(model.affinity_matrix_)

distances, indices = knn.kneighbors(dist.pairwise(y.T, y_pred.T))

labels = stats.mode(model.labels_[indices], axis=1)[0].flatten()

sns.scatterplot(x=y[0], y=y[1], hue=labels)

In [ ]:
# Colonnes dont on a besoin. Pour rappel nos colonnes scalées sont :
# ['amount_log_std_mm', 'number_of_orders_log_std_mm', 'recency_std_mm', 'respected_ratio_std_mm', 'freight_value_log_std_mm']
log_selected = ["amount", "number_of_orders", "freight_value"]
# On selectionne toutes les données pour le standard scaler et le minmax
selected = ["amount", "number_of_orders", "recency", "respected_ratio", "freight_value"]

# On conserve les données dont on a besoin uniquement
df = customers[selected].copy()

# On scale les données
# LOG
def log_scale(x):
    return np.log(x+1e-8)
df[log_selected] = df[log_selected].apply(log_scale)

# STD (selected == df.columns)
std = StandardScaler()
df[selected] = std.fit_transform(df)

# MM
mm = MinMaxScaler()
df[selected] = mm.fit_transform(df)

In [ ]:
def preprocess(X):
    X[log_selected] = log_scale(X[log_selected])
    X[selected] = std.transform(X)
    X[selected] = mm.transform(X)
    return X

In [ ]:
df[selected].describe()

In [ ]:
# Projection
projector = umap.UMAP(n_neighbors=50, random_state=0, n_jobs=-1, repulsion_strength=1.2)
projector.fit(df)
y = projector.embedding_

In [ ]:
sns.scatterplot(x=y.T[0], y=y.T[1], hue=customers["segment"]);

In [ ]:
# Pour l'ARI on doit recalculer à chaque fois le SpectralClustering et notre calcul de neighbor (évidemment sinon c'est de la triche !)
def model_f(X, n_spectral=None, verbose=True):
    if(n_spectral is None):
        n_spectral = int(X.shape[0] * 0.008)
        if verbose: print(f"Using {n_spectral=}")
    # Random pick des n_spectral
    ids = np.random.choice(X.shape[0], n_spectral, replace=False)
    X_spectral = X[ids]

    # SC
    if verbose: print("Fitting SpectralClustering")
    model = SpectralClustering(n_clusters=4, n_jobs=-1)
    model.fit(X_spectral)

    # NN
    if verbose: print("Calculating NearestNeibors from samples")
    dist = DistanceMetric.get_metric('euclidean')
    knn = NearestNeighbors(n_neighbors=3, metric='precomputed')
    knn.fit(model.affinity_matrix_)

    # Calcul de distance moins lourd - récupération des indexes des neighbors
    indices = knn.kneighbors(dist.pairwise(X, X_spectral))[1]

    # Labels
    if verbose: print("Getting labels")
    labels = stats.mode(model.labels_[indices], axis=1, keepdims=False)[0].flatten()
    if verbose: 
        sns.scatterplot(x=X.T[0], y=X.T[1], hue=labels)
        plt.show()

    return labels

In [ ]:
df_true_labels = model_f(y)

In [ ]:
scores = []

tqdm_b = tqdm(range(40, 20*30, 10))
for d in tqdm_b:
    try:
        # On perd des clients qu'on avait auparavant
        data_m = get_data_until(days=d)

        # Les clients correspondant dans 'df'
        df_m = df[customers.customer_unique_id.isin(data_m.customer_unique_id)]
        df_labels = df_true_labels[df_m.index]

        # Preprocess
        data_m = preprocess(data_m[selected].copy())
        
        emb = projector.transform(data_m)

        labels = model_f(emb, verbose=False)

        # On récupère le score ARI
        ari = adjusted_rand_score(df_labels, labels)
        tqdm_b.set_postfix({"score": ari})
        scores.append(ari)
    except Exception as e:
        print(f"Error at {d=}", e)

scores

In [ ]:
plt.figure(figsize=(16,9))
plt.title("ARI score per day difference")
ax.yaxis.set_ticks(np.arange(-2, 2, 0.25))
plt.plot(range(40, 20*30, 10), scores)